In [1]:
#Dependencies
import os
import urllib
import json
import csv
import simplejson
import pandas as pd
import pytest
import base64
import requests
from pprint import pprint
from datetime import datetime
from ohmysportsfeedspy.v1_0 import API_v1_0
from ohmysportsfeedspy.v1_1 import API_v1_1
from ohmysportsfeedspy.v1_2 import API_v1_2
from ohmysportsfeedspy import MySportsFeeds
msf = MySportsFeeds(version="1.2")

In [209]:
NFL_data = "box_scores.csv"
NFL_data = pd.read_csv(NFL_data)
NFL_data.head(1)

In [208]:
team_scores = NFL_data[["date","visitor","home","visitor_score","home_score"]]
team_scores.head(1)

In [81]:
NFL_games = "nfl_games.csv"
NFL_games = pd.read_csv(NFL_games)
NFL_date_games = NFL_games.set_index("date")

NFL_date_games = NFL_date_games["2014-09-04":]

In [207]:
NFL_date_games = NFL_date_games[["season","team1","team2","score1","score2"]]
nfl_scores = NFL_date_games
nfl_scores.head()

In [122]:
team1_scores = nfl_score.groupby("team1").sum()["score1"]
team2_scores = nfl_score.groupby("team2").sum()["score2"]

team1_scores_df = pd.DataFrame(team1_scores)
team2_scores_df = pd.DataFrame(team2_scores)

In [206]:
#Scores from 2014 to 2017
nfl_team_scores = pd.concat([team1_scores_df, team2_scores_df], axis=1)
nfl_team_scores["Total Points from 2014 to 2017"] = nfl_team_scores["score1"] + nfl_team_scores["score2"]
del nfl_team_scores["score1"]
del nfl_team_scores["score2"]
nfl_team_scores.head()

In [ ]:
# Obtained all scores for season, including full season, called DataFrames on all.

nfl_season_2014 = NFL_date_games["2014-09-04":"2015-02-01"]
scores_2014_team1 = nfl_season_2014.groupby("team1").sum()["score1"]
scores_2014_team2 = nfl_season_2014.groupby("team2").sum()["score2"]
scores_2014_df1 = pd.DataFrame(scores_2014_team1)
scores_2014_df2 = pd.DataFrame(scores_2014_team2)
scores_2014 = pd.concat([scores_2014_df1, scores_2014_df2], axis = 1)
scores_2014["Total Points for 2014"] = scores_2014["score1"] + scores_2014["score2"]
del scores_2014["score1"]
del scores_2014["score2"]

nfl_season_2015 = NFL_date_games["2015-09-10":"2016-02-07"]
scores_2015_team1 = nfl_season_2015.groupby("team1").sum()["score1"]
scores_2015_team2 = nfl_season_2015.groupby("team2").sum()["score2"]
scores_2015_df1 = pd.DataFrame(scores_2015_team1)
scores_2015_df2 = pd.DataFrame(scores_2015_team2)
scores_2015 = pd.concat([scores_2015_df1, scores_2015_df2], axis = 1)
scores_2015["Total Points for 2015"] = scores_2015["score1"] + scores_2015["score2"]
del scores_2015["score1"]
del scores_2015["score2"]


nfl_season_2016 = NFL_date_games["2016-09-08":"2017-02-05"]
scores_2016_team1 = nfl_season_2016.groupby("team1").sum()["score1"]
scores_2016_team2 = nfl_season_2016.groupby("team2").sum()["score2"]
scores_2016_df1 = pd.DataFrame(scores_2016_team1)
scores_2016_df2 = pd.DataFrame(scores_2016_team2)
scores_2016 = pd.concat([scores_2016_df1, scores_2016_df2], axis = 1)
scores_2016["Total Points for 2016"] = scores_2016["score1"] + scores_2016["score2"]
del scores_2016["score1"]
del scores_2016["score2"]


nfl_team_scores
scores_2014
scores_2015
scores_2016

In [241]:
#Getting final row for sums for each DF

nfl_team_scores_sum = nfl_team_scores.loc['Total', 'Sum of Point']= nfl_team_scores['Total Points from 2014 to 2017'].sum()
scores_2014_sum = scores_2014.loc['Total', 'Sum of Point']= scores_2014['Total Points for 2014'].sum()
scores_2015_sum = scores_2015.loc['Total', 'Sum of Point']= scores_2015['Total Points for 2015'].sum()
scores_2016_sum = scores_2016.loc['Total', 'Sum of Point']= scores_2016['Total Points for 2016'].sum()

In [247]:
sum_all = nfl_team_scores["Total Points from 2014 to 2017"].sum()